In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'3ophprxt'

In [3]:
wandb.init(id='2rrh11ch', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 1min 4s, sys: 2.38 s, total: 1min 6s
Wall time: 1min 6s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
set_lr(3e-4)

In [11]:
model_stem = "9.21"
START_E = 21

In [12]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = get_lr() #6e-4 if e < 40 else 3e-4 if e < 80 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 21

{'trn_control_loss': 0.04192579, 'trn_headings_loss': 0.00208092, 'trn_curvatures_loss': 0.00597745, 'consistency losses/trn_steer_cost': 0.00913881, 'consistency losses/trn_te_loss': 23.27855118, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 3945.86565208, 'consistency losses/trn_torque_delta_loss': 28.55055542, 'logistical/obs_consumed_per_second': 196.91338583, 'logistical/obs_generated_per_second': 164.28346457, 'logistical/data_consumption_ratio': 1.19941263, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.26980972, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 199.375, 'logistical/obs_generated_per_second': 166.515625, 'logistical/data_consumption_ratio': 1.19747122, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0353182, 'trn_headings_loss': 0.00188011, 'trn_curvatures_loss': 0.00490578, 'consistency losses/trn_steer_cost': 0.00715619, 'consistency losses/trn_te_lo

{'logistical/obs_consumed_per_second': 200.296875, 'logistical/obs_generated_per_second': 171.0546875, 'logistical/data_consumption_ratio': 1.17124675, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0266754, 'trn_headings_loss': 0.00134704, 'trn_curvatures_loss': 0.00317252, 'consistency losses/trn_steer_cost': 0.00543854, 'consistency losses/trn_te_loss': 24.02041616, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 4241.14766884, 'consistency losses/trn_torque_delta_loss': 25.17050446, 'logistical/max_param': 16.26283646, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 195.7578125, 'logistical/obs_generated_per_second': 167.9765625, 'logistical/data_consumption_ratio': 1.16544493, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03625065, 'trn_headings_loss': 0.00180854, 'trn_curvatures_loss': 0.00434557, 'consistency losses/trn_steer_cost': 0.00677396, 'consistency losses/trn_te_loss': 23

{'logistical/obs_consumed_per_second': 195.90625, 'logistical/obs_generated_per_second': 169.453125, 'logistical/data_consumption_ratio': 1.15642202, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03007931, 'trn_headings_loss': 0.00165176, 'trn_curvatures_loss': 0.00429916, 'consistency losses/trn_steer_cost': 0.00676879, 'consistency losses/trn_te_loss': 23.42851003, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 4243.86701107, 'consistency losses/trn_torque_delta_loss': 25.42342344, 'logistical/max_param': 16.25957108, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 203.65625, 'logistical/obs_generated_per_second': 169.953125, 'logistical/data_consumption_ratio': 1.19892464, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03606606, 'trn_headings_loss': 0.0017596, 'trn_curvatures_loss': 0.00604218, 'consistency losses/trn_steer_cost': 0.00870782, 'consistency losses/trn_te_loss': 22.81749

{'logistical/obs_consumed_per_second': 198.5703125, 'logistical/obs_generated_per_second': 167.46875, 'logistical/data_consumption_ratio': 1.18512384, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02307922, 'trn_headings_loss': 0.00119177, 'trn_curvatures_loss': 0.00325204, 'consistency losses/trn_steer_cost': 0.00558247, 'consistency losses/trn_te_loss': 23.02540223, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 4485.39767838, 'consistency losses/trn_torque_delta_loss': 23.0553328, 'logistical/max_param': 16.26416397, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 150.0, 'logistical/obs_generated_per_second': 168.0, 'logistical/data_consumption_ratio': 0.89428898, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 25

{'trn_control_loss': 0.0325309, 'trn_headings_loss': 0.00178866, 'trn_curvatures_loss': 0.00588499, 'consistency losses/trn_steer_cost': 0.00702664, 'consistency losses/trn_te_l

{'logistical/obs_consumed_per_second': 201.1640625, 'logistical/obs_generated_per_second': 167.828125, 'logistical/data_consumption_ratio': 1.19819185, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03754777, 'trn_headings_loss': 0.00174497, 'trn_curvatures_loss': 0.00577514, 'consistency losses/trn_steer_cost': 0.00901411, 'consistency losses/trn_te_loss': 22.69081149, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 4039.29720592, 'consistency losses/trn_torque_delta_loss': 22.69092436, 'logistical/max_param': 16.25802803, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 199.125, 'logistical/obs_generated_per_second': 167.7578125, 'logistical/data_consumption_ratio': 1.18773422, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02966487, 'trn_headings_loss': 0.00139994, 'trn_curvatures_loss': 0.00483052, 'consistency losses/trn_steer_cost': 0.00618234, 'consistency losses/trn_te_loss': 24.10

...

{'logistical/obs_consumed_per_second': 193.5390625, 'logistical/obs_generated_per_second': 165.4375, 'logistical/data_consumption_ratio': 1.17036831, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01655519, 'trn_headings_loss': 0.0007339, 'trn_curvatures_loss': 0.00266168, 'consistency losses/trn_steer_cost': 0.00713439, 'consistency losses/trn_te_loss': 22.9569588, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 4326.13160896, 'consistency losses/trn_torque_delta_loss': 22.48497496, 'logistical/max_param': 16.26135826, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 198.140625, 'logistical/obs_generated_per_second': 165.671875, 'logistical/data_consumption_ratio': 1.1963355, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00999495, 'trn_headings_loss': 0.00045462, 'trn_curvatures_loss': 0.00152859, 'consistency losses/trn_steer_cost': 0.0047228, 'consistency losses/trn_te_loss': 23.03129406

{'logistical/obs_consumed_per_second': 200.78125, 'logistical/obs_generated_per_second': 164.671875, 'logistical/data_consumption_ratio': 1.21853018, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01313412, 'trn_headings_loss': 0.0006399, 'trn_curvatures_loss': 0.00248559, 'consistency losses/trn_steer_cost': 0.00503671, 'consistency losses/trn_te_loss': 23.13094302, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9609375, 'consistency losses/trn_torque_loss': 4280.2701025, 'consistency losses/trn_torque_delta_loss': 21.54820748, 'logistical/max_param': 16.26160622, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 197.21875, 'logistical/obs_generated_per_second': 163.7734375, 'logistical/data_consumption_ratio': 1.20433049, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01164392, 'trn_headings_loss': 0.00062813, 'trn_curvatures_loss': 0.0024962, 'consistency losses/trn_steer_cost': 0.00594337, 'consistency losses/trn_te_loss': 22.79200

{'logistical/obs_consumed_per_second': 194.4765625, 'logistical/obs_generated_per_second': 155.46875, 'logistical/data_consumption_ratio': 1.25132831, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01105313, 'trn_headings_loss': 0.00045282, 'trn_curvatures_loss': 0.00158047, 'consistency losses/trn_steer_cost': 0.005108, 'consistency losses/trn_te_loss': 23.92022443, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.859375, 'consistency losses/trn_torque_loss': 3791.22395229, 'consistency losses/trn_torque_delta_loss': 19.2863717, 'logistical/max_param': 16.25529099, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 198.609375, 'logistical/obs_generated_per_second': 163.4765625, 'logistical/data_consumption_ratio': 1.21527196, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01675972, 'trn_headings_loss': 0.0007431, 'trn_curvatures_loss': 0.00275834, 'consistency losses/trn_steer_cost': 0.0057245, 'consistency losses/trn_te_loss': 22.721824

{'logistical/obs_consumed_per_second': 198.0703125, 'logistical/obs_generated_per_second': 167.46875, 'logistical/data_consumption_ratio': 1.18355, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01010543, 'trn_headings_loss': 0.00055379, 'trn_curvatures_loss': 0.00155971, 'consistency losses/trn_steer_cost': 0.00395024, 'consistency losses/trn_te_loss': 23.5950883, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 4643.84889984, 'consistency losses/trn_torque_delta_loss': 22.68164125, 'logistical/max_param': 16.26430702, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 201.0703125, 'logistical/obs_generated_per_second': 164.40625, 'logistical/data_consumption_ratio': 1.22263665, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01152113, 'trn_headings_loss': 0.00063441, 'trn_curvatures_loss': 0.0020141, 'consistency losses/trn_steer_cost': 0.00450623, 'consistency losses/trn_te_loss': 23.416092

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e95.torch")

In [14]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
